# Export podataka za GeNIe Bayesovu mrežu

**Cilj:** Kreirati CSV datoteku s diskretiziranim varijablama za import u GeNIe.

**Ulazni podaci:**
- `../dataset/student-por.csv` - originalni dataset
- `../drugi/WoE_IV_rezultati.csv` - IV vrijednosti za identifikaciju prediktora

**Izlaz:**
- `student_genie.csv` - diskretiziran dataset za GeNIe

**Interpretacija IV vrijednosti:**
| IV vrijednost | Snaga prediktora |
|---|---|
| < 0.02 | Varijabla nije korisna |
| 0.02 – 0.1 | Slaba prediktivnost |
| 0.1 – 0.3 | Srednja prediktivnost |
| 0.3 – 0.5 | Jaka prediktivnost |
| > 0.5 | Super prediktivnost |

In [1]:
import pandas as pd
import numpy as np

print("Biblioteke učitane.")

Biblioteke učitane.


In [2]:
# Učitaj IV rezultate iz prethodne analize
iv_df = pd.read_csv('../drugi/WoE_IV_rezultati.csv', sep=';')

print("IV REZULTATI IZ WoE ANALIZE:")
print("="*60)
display(iv_df)

IV REZULTATI IZ WoE ANALIZE:


,Varijabla,IV,Prediktivnost
0,failures,0.8696,Super prediktivnost
1,higher,0.5162,Super prediktivnost
2,school,0.4156,Jaka prediktivnost
3,Medu,0.2648,Srednja prediktivnost
4,age,0.2360,Srednja prediktivnost
5,studytime,0.2147,Srednja prediktivnost
6,Dalc,0.2017,Srednja prediktivnost
7,Mjob,0.1797,Srednja prediktivnost
8,Fedu,0.1600,Srednja prediktivnost
9,reason,0.1543,Srednja prediktivnost


In [3]:
# Kreiranje dataseta za GeNIe s diskretiziranim varijablama

print("PRIPREMA PODATAKA ZA GENIE BAYESOVU MREŽU")
print("="*80)

# Učitaj originalne podatke
df_genie = pd.read_csv('../dataset/student-por.csv', sep=';')

# Dodaj ciljnu varijablu
df_genie['target_G1'] = (df_genie['G1'] <= 10).astype(int)

# Lista varijabli za export (bez G1, G2, G3 - jer su povezane s targetom)
exclude_from_export = ['G1', 'G2', 'G3']

def discretize_variable(series, n_bins=5):
    """Diskretizira kontinuiranu varijablu u kategorije."""
    if series.nunique() <= 10:
        return series.astype(str)
    else:
        try:
            binned = pd.qcut(series, n_bins, duplicates='drop')
            return binned.astype(str)
        except:
            binned = pd.cut(series, n_bins, duplicates='drop')
            return binned.astype(str)

# Primijeni diskretizaciju na numeričke varijable
numeric_cols = df_genie.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in exclude_from_export + ['target_G1']]

print(f"\nDiskretizacija numeričkih varijabli:")
for col in numeric_cols:
    n_unique = df_genie[col].nunique()
    if n_unique > 10:
        df_genie[col] = discretize_variable(df_genie[col])
        print(f"  {col}: {n_unique} → diskretiziran u kategorije")
    else:
        df_genie[col] = df_genie[col].astype(str)
        print(f"  {col}: {n_unique} vrijednosti (zadržano)")

# Ukloni G1, G2, G3 iz exporta
df_export = df_genie.drop(columns=exclude_from_export)

# Provjeri nazive stupaca (GeNIe zahtijeva validne nazive)
df_export.columns = [c.replace(' ', '_').replace('-', '_') for c in df_export.columns]

print(f"\n{'='*80}")
print(f"KONAČNI DATASET ZA GENIE:")
print(f"  - Broj učenika (redaka): {len(df_export)}")
print(f"  - Broj varijabli (stupaca): {len(df_export.columns)}")
print(f"  - Varijable: {list(df_export.columns)}")

print(f"\nPRVIH 5 REDAKA:")
display(df_export.head())

# Export u CSV
output_path = 'student_genie.csv'
df_export.to_csv(output_path, index=False)
print(f"\n Dataset uspješno exportan u '{output_path}'")

PRIPREMA PODATAKA ZA GENIE BAYESOVU MREŽU

Diskretizacija numeričkih varijabli:
  age: 8 vrijednosti (zadržano)
  Medu: 5 vrijednosti (zadržano)
  Fedu: 5 vrijednosti (zadržano)
  traveltime: 4 vrijednosti (zadržano)
  studytime: 4 vrijednosti (zadržano)
  failures: 4 vrijednosti (zadržano)
  famrel: 5 vrijednosti (zadržano)
  freetime: 5 vrijednosti (zadržano)
  goout: 5 vrijednosti (zadržano)
  Dalc: 5 vrijednosti (zadržano)
  Walc: 5 vrijednosti (zadržano)
  health: 5 vrijednosti (zadržano)
  absences: 24 → diskretiziran u kategorije

KONAČNI DATASET ZA GENIE:
  - Broj učenika (redaka): 649
  - Broj varijabli (stupaca): 31
  - Varijable: ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'target_G1']

PRVIH 5 REDAKA:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,target_G1
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,"(2.0, 4.0]",1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,"(-0.001, 2.0]",1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,"(4.0, 6.0]",0
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,"(-0.001, 2.0]",0
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,"(-0.001, 2.0]",0



 Dataset uspješno exportan u 'student_genie.csv'


In [4]:
# Sažetak za GeNIe - koje varijable trebaju forsirane veze

print("UPUTE ZA GENIE - STRUKTURA BAYESOVE MREŽE")
print("="*80)

# Interpretacija IV vrijednosti:
# < 0.02 - Varijabla nije korisna
# 0.02 - 0.1 - Slaba prediktivnost
# 0.1 - 0.3 - Srednja prediktivnost
# 0.3 - 0.5 - Jaka prediktivnost
# > 0.5 - Super prediktivnost

# Podijeli varijable po kategorijama
super_predictors = iv_df[iv_df['IV'] > 0.5]['Varijabla'].tolist()
strong_predictors = iv_df[(iv_df['IV'] >= 0.3) & (iv_df['IV'] <= 0.5)]['Varijabla'].tolist()
medium_predictors = iv_df[(iv_df['IV'] >= 0.1) & (iv_df['IV'] < 0.3)]['Varijabla'].tolist()
weak_predictors = iv_df[(iv_df['IV'] >= 0.02) & (iv_df['IV'] < 0.1)]['Varijabla'].tolist()
useless_predictors = iv_df[iv_df['IV'] < 0.02]['Varijabla'].tolist()

print("\n SUPER PREDIKTIVNOST (IV > 0.5) - FORSIRANA VEZA NA target_G1:")
print("-"*60)
for i, var in enumerate(super_predictors, 1):
    iv_val = iv_df[iv_df['Varijabla'] == var]['IV'].values[0]
    print(f"  {i:2d}. {var} (IV = {iv_val:.4f})")

print("\n JAKA PREDIKTIVNOST (IV 0.3-0.5) - FORSIRANA VEZA NA target_G1:")
print("-"*60)
for i, var in enumerate(strong_predictors, 1):
    iv_val = iv_df[iv_df['Varijabla'] == var]['IV'].values[0]
    print(f"  {i:2d}. {var} (IV = {iv_val:.4f})")

print("\n SREDNJA PREDIKTIVNOST (IV 0.1-0.3) - ALGORITAM UČI VEZE:")
print("-"*60)
for i, var in enumerate(medium_predictors, 1):
    iv_val = iv_df[iv_df['Varijabla'] == var]['IV'].values[0]
    print(f"  {i:2d}. {var} (IV = {iv_val:.4f})")

print("\n SLABA PREDIKTIVNOST (IV 0.02-0.1) - ALGORITAM UČI VEZE:")
print("-"*60)
for i, var in enumerate(weak_predictors, 1):
    iv_val = iv_df[iv_df['Varijabla'] == var]['IV'].values[0]
    print(f"  {i:2d}. {var} (IV = {iv_val:.4f})")

print("\n NIJE KORISNA (IV < 0.02):")
print("-"*60)
for i, var in enumerate(useless_predictors, 1):
    iv_val = iv_df[iv_df['Varijabla'] == var]['IV'].values[0]
    print(f"  {i:2d}. {var} (IV = {iv_val:.4f})")

print(f"\n{'='*80}")
print("KORACI U GENIE:")
print("  1. File → Open Data File → odaberi 'student_genie.csv'")
print("  2. Za SUPER i JAKE prediktore: ručno dodaj arc prema 'target_G1'")
print("  3. Learning → Learn Structure → odaberi algoritam (npr. PC, K2)")
print("  4. Algoritam će naučiti veze za ostale prediktore")
print("="*80)

UPUTE ZA GENIE - STRUKTURA BAYESOVE MREŽE

 SUPER PREDIKTIVNOST (IV > 0.5) - FORSIRANA VEZA NA target_G1:
------------------------------------------------------------
   1. failures (IV = 0.8696)
   2. higher (IV = 0.5162)

 JAKA PREDIKTIVNOST (IV 0.3-0.5) - FORSIRANA VEZA NA target_G1:
------------------------------------------------------------
   1. school (IV = 0.4156)

 SREDNJA PREDIKTIVNOST (IV 0.1-0.3) - ALGORITAM UČI VEZE:
------------------------------------------------------------
   1. Medu (IV = 0.2648)
   2. age (IV = 0.2360)
   3. studytime (IV = 0.2147)
   4. Dalc (IV = 0.2017)
   5. Mjob (IV = 0.1797)
   6. Fedu (IV = 0.1600)
   7. reason (IV = 0.1543)
   8. traveltime (IV = 0.1311)
   9. address (IV = 0.1177)
  10. absences (IV = 0.1078)
  11. Walc (IV = 0.1072)

 SLABA PREDIKTIVNOST (IV 0.02-0.1) - ALGORITAM UČI VEZE:
------------------------------------------------------------
   1. famrel (IV = 0.0975)
   2. guardian (IV = 0.0933)
   3. freetime (IV = 0.0932)
   4. 